Detección de mascatillas mediante la combinación de dos CNN. Primero, la imagen se segmenta utilizando BodyPix de Google y se crean máscaras a partir de las caras. A continuación, se determina una ROI con cada máscara y se proporciona como input al modelo clasificador. Finalmente, se pinta en la imagen original la ROI con la etiqueta clasificada.

# Montar el Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
os.chdir('/content/gdrive/My Drive/TFM-MaskDetection/classification-models/') # Cambia al directorio donde se encuentra training.ipynb

# Descargar BodyPix

In [ ]:
pip install tf-bodypix[all]

# Test de BodyPix

In [ ]:
_SAVE = False     # Guardar el resultado final

IMAGE_SRC_PATH = "Test/src/"
IMAGE_MASK_PATH = "Test/mask/"
IMAGE_DST_PATH = "Test/dst/"
IMAGE_NAME = "test2"
IMAGE_EXTENSION = ".jpg"

IMAGE_SRC = IMAGE_SRC_PATH + IMAGE_NAME + IMAGE_EXTENSION
IMAGE_DST_MASK = IMAGE_MASK_PATH + IMAGE_NAME + "_mask" + IMAGE_EXTENSION
IMAGE_DST_COLOR = IMAGE_MASK_PATH + IMAGE_NAME + "_color" + IMAGE_EXTENSION
IMAGE_DST_FACE = IMAGE_MASK_PATH + IMAGE_NAME + "_face" + IMAGE_EXTENSION

In [ ]:
import tensorflow as tf
from tf_bodypix.api import download_model, load_model, BodyPixModelPaths

import numpy as np
import matplotlib.pyplot as plt

def check_binary_mask(mat):
  x = np.count_nonzero((mat!=0) & (mat!=1))
  return (x == 0)

bodypix_model = load_model(
    model_path=download_model(BodyPixModelPaths.MOBILENET_FLOAT_100_STRIDE_16)
    )

image = tf.keras.preprocessing.image.load_img(IMAGE_SRC)
image_array = tf.keras.preprocessing.image.img_to_array(image)
result = bodypix_model.predict_single(image_array)
mask = result.get_mask(threshold=0.3)
tf.keras.preprocessing.image.save_img(IMAGE_DST_MASK, mask)

colored_mask = result.get_colored_part_mask(mask)
tf.keras.preprocessing.image.save_img(IMAGE_DST_COLOR, colored_mask)

face_mask = result.get_part_mask(mask, ['left_face','right_face'])
tf.keras.preprocessing.image.save_img(IMAGE_DST_FACE, face_mask)

In [ ]:
from PIL import Image
import cv2 as cv

def createROI(mask):
  # Dilate mask
  kernel = np.ones((5,5), np.uint8)
  mask = cv.erode(mask, kernel, iterations=10)
  mask = cv.dilate(mask, kernel, iterations=10)

  """
  plt.imshow(mask, cmap='gray')
  plt.axis('off')
  plt.rcParams['figure.figsize'] = [12, 8]
  plt.rcParams['figure.dpi'] = 100
  plt.savefig(IMAGE_DST_PATH + IMAGE_NAME + '_maskprocess' + IMAGE_EXTENSION, bbox_inches='tight')
  """

  # Create ROI
  contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
  contours_poly = [None]*len(contours)
  boundRect = [None]*len(contours)
  for i, c in enumerate(contours):
    contours_poly[i] = cv.approxPolyDP(c, 3, True)
    boundRect[i] = cv.boundingRect(contours_poly[i])
  return boundRect

def drawROI(boundRect, img_dst):
  for i in range(boundRect.shape[0]):
    cv.rectangle(img_dst, (int(boundRect[i][0]), int(boundRect[i][1])), \
                (int(boundRect[i][0]+boundRect[i][2]), \
                int(boundRect[i][1]+boundRect[i][3])), (255,0,0), 2)


img_src = cv.imread(IMAGE_SRC)  # Source image
mask = cv.imread(IMAGE_DST_FACE, cv.CV_8U); # Mask image
ROIs = createROI(mask)
img_src = cv.cvtColor(img_src, cv.COLOR_BGR2RGB)

# Plot en la imagen original
img_dst = img_src
for i, r in enumerate(ROIs):
  ymin, xmin, h, w = r
  ymax = ymin + h
  xmax = xmin + w
  img_dst = cv.rectangle(img_dst, (ymin,xmin), (ymax,xmax), (0,255,0), 2)

plt.imshow(img_dst)
plt.axis('off')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100
if _SAVE:
  plt.savefig(IMAGE_DST_PATH + IMAGE_NAME + IMAGE_EXTENSION, bbox_inches='tight')
  print("Image saved in " + IMAGE_DST_PATH + IMAGE_NAME + IMAGE_EXTENSION)
plt.show()


# Inferencia conjunta

In [ ]:
import os
import glob
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv2
from tf_bodypix.api import download_model, load_model, BodyPixModelPaths

TRAINING_DATA_DIR = r"./dataset"
MODEL_NAME = r"mobilenet_v2/100"
SAVED_PATH = os.path.join("saved_models", MODEL_NAME)
IMAGE_SRC_FOLDER = r"Test/src/"
IMAGE_DST_FOLDER = r"Test/dst/"
IMAGE_EXTENSION = r".png"

def createROI(mask):
    # Dilate mask
    kernel = np.ones((5,5), np.uint8)
    mask = cv.erode(mask, kernel, iterations=3)
    mask = cv.dilate(mask, kernel, iterations=3)
    
    # Create ROI
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, 
                        cv.CHAIN_APPROX_NONE)
    contours_poly = [None]*len(contours)
    bbox = [None]*len(contours)
    for i, c in enumerate(contours):
        contours_poly[i] = cv.approxPolyDP(c, 3, True)
        bbox[i] = cv.boundingRect(contours_poly[i])
    return bbox
    
# Download models
bodypix_model = load_model(
    model_path=download_model(
    BodyPixModelPaths.MOBILENET_FLOAT_100_STRIDE_16)
)   # Segmentation model
model = tf.keras.models.load_model(SAVED_PATH) # Classification model
dataset_labels = np.array(
    ['Mask_Weared_Incorrect', 'With_Mask', 'Without_Mask'])
color_labels = [(0,0,255), (0,255,0), (255,0,0)]

img_list = glob.glob(IMAGE_SRC_FOLDER + '*' + IMAGE_EXTENSION)
for n, img_src in enumerate(img_list):
    print(n, ":", img_src)
    # Load image
    img = tf.keras.preprocessing.image.load_img(img_src)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    
    # Apply BodyPixModelPaths
    retult = bodypix_model.predict_single(img_array)
    mask = result.get_mask(threshold=0.6)
    # Create mask and calculate ROIs
    face_mask = result.get_part_mask(mask, 
        ['left_face', 'right_face'])
    ROIs = createROI(face_mask.astype(np.uint8))
    
    if (len(ROIs) > 0):
        # Create image batch based on ROIs
        img_batch = np.empty((len(ROIs), 224, 224, 3), 
            dtype.np.float32)
        for i, r in enumerate(ROIs):
            ymin, xmin, h, w = r
            ymax = ymin + h
            xmax = xmin + w
            img_roi = np.zeros([w, h, 3], dtype=np.float32)
            img_roi = img_array([xmin:xmax, ymin:ymax, :])
            img_roi = cv.resize(img_roi, (224,224))
            img_batch[i, ...] = img_roi / 255.0
        
        # Classify image batch
        classif = model.predict(img_batch, batch_size=len(ROIs))
        classif_ids = np.argmax(classif, axis=-1)
        classif_labels = dataset_labels[classif_ids]
        
        # Create destination image
        img_dst = img_array.astype(np.uint8)
        for i, r in enumerate(ROIs):
            ymin, xmin, h, w = r
            ymax = ymin + h
            xmax = xmin + w
            color = color_labels[classif_ids[i]]
            img_dst = cv.rectangle(img_dst, (ymin,xmin), 
                        (ymax, xmax), color, 2)
            
        # Save image into disk
        plt.imshow(img_dst)
        plt.axis('off')
        plt.savefig(IMAGE_DST_FOLDER + str(n), bbox_inches='tight')